# Uso del api de OpenAI
---
<style>
      h1, h2, h3, h4, h5, h6,.imagen {
        text-align: center;
      }
 img{width: 75%; height: 75%;}
</style>

Para utilizar la API de OpenAI, se necesita una clave de API. Para obtenerla, debe registrarse en OpenAI. Estos servicios tienen un costo por su uso, pero son económicos. Además, al principio, ofrecen una cantidad para que el usuario pueda probar la API.

## Gradio

[Gradio](https://www.gradio.app/) es una biblioteca que permite crear interfaces de usuario. Estas interfaces se pueden utilizar para probar modelos de aprendizaje automático. 



## Configuración

```python
#instalar librerias
%pip install openai # Instalar la librería de OpenAI
%pip install python-dotenv # Instalar la librería para leer el archivo .env
%pip install ipywidgets # Instalar la librería para mostrar widgets
%pip install ipython # Instalar la librería para mostrar widgets
%pip install gradio # Instalar la librería para mostrar  interfaz gráfica
%pip install pillow # manejar imagenes
%pip install python-pptx # poder crear pptx
```


```python
#crear secretos
import os
from dotenv import load_dotenv
import ipywidgets
from IPython.display import clear_output


#crear archivo .env y agregar la variable de entorno
valor  =input("Ingresa tu API key de OpenAI: ")
clear_output()
if os.path.exists(".env"):
    secretos = open(".env", "a")
else:
    secretos = open(".env", "w")
secretos.write(f"OPENAI_API_KEY={valor}")
secretos.close()
valor = ""
load_dotenv()

```

## Consulta básica

```python
#Consulta a la API del chat
import openai
import time

key = os.getenv("OPENAI_API_KEY")

openai.api_key = key

MensajeDSistema = "mensaje de sistema"#@param {type:"string"}
consulta = "consulta"#@param {type:"string"}
temperatura = 0.9 #@param {type:"slider", min:0.1, max:2, step:0.1}

mensajes = [{"role": "system", "content": MensajeDSistema},
            {"role": "user", "content": consulta} ]

def LLM( mensajes, temp):
    try:
        resultado = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
          messages=mensajes,
          temperature=temp)
    except Exception as e :
        time.sleep(5)
        print(f"Error de conexión: {e}")
        LLM(mensajes, temp)
    return resultado["choices"][0]["message"]["content"]

LLM(mensajes=mensajes,temp=temperatura)
```

## Chatbot

```python
#chatbot basico
import openai
import gradio as gr
import os
import time

# Obtiene la clave de API desde las variables de entorno
key = os.getenv("OPENAI_API_KEY")

# Asigna la clave de API a la variable api_key del módulo openai
openai.api_key = key

# Crea una lista vacía para almacenar los mensajes de la conversación
mensajes = []

system_msg = "Eres un asistente util"#@param {type: "string"}
mensajes.append({"role": "system", "content": system_msg})


def LLM(mensaje):
    try:
        resultado = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=mensajes)
    except  Exception as e:
        time.sleep(5)

        print(f"Error de conexión: {e}")
        LLM(mensaje)
    return resultado["choices"][0]["message"]["content"]

def chat(mensaje,historial):
        # Agrega el mensaje del usuario a la lista de mensajes
    mensajes.append({"role": "user", "content": mensaje})

        # Utiliza la función ChatCompletion de la API de OpenAI para generar una respuesta del bot
    respuesta = LLM(mensaje)

        # Obtiene la respuesta generada por el bot y la agrega a la lista de mensajes

    mensajes.append({"role": "assistant", "content": respuesta})

    historial.append((f"Usuario:\n{mensaje}", f"Asistente:\n{respuesta}"))
    return  "",historial

with gr.Blocks() as interfaz:
    chatbot = gr.Chatbot()
    entrada=gr.Textbox()
    limpiar=gr.ClearButton([entrada,chatbot])
    entrada.submit(chat,[entrada,chatbot],[entrada,chatbot])
interfaz.launch()

```

## Texto a Imagenes

```python
# Generar imágenes con OpenAI

import openai
import time
from PIL import Image
import requests
# Obtiene la clave de API desde las variables de entorno
key = os.getenv("OPENAI_API_KEY")

# Asigna la clave de API a la variable api_key del módulo openai
openai.api_key = key

prompt = "Una imagen de un gato al estilo manga y cyberpunk"#@param {type:"string"}
size = "256x256" #@param [ "256x256", "512x512", "1024x1024"]

def GenerarImagen(prompt,size="256x256"):
    try:
        respuesta=  openai.Image.create(
            prompt=prompt,
            n=1,
            size=size)
    except Exception as e:
        time.sleep(5)
        print(f"Error de conexión: {e}")
        GenerarImagen(prompt,size)

    url=respuesta['data'][0]['url']
    imagen=Image.open(requests.get(url, stream=True).raw)
    return imagen


GenerarImagen(prompt,size)
```

# Generador de imagen con gui

```python
# Generador de imagen con gui
import openai
import time
import gradio as gr

# Obtiene la clave de API desde las variables de entorno
key = os.getenv("OPENAI_API_KEY")

# Asigna la clave de API a la variable api_key del módulo openai
openai.api_key = key

def GenerarImagen(prompt,size="512x512"):
    try:
        respuesta=  openai.Image.create(
            prompt=prompt,
            n=1,
            size=size)
    except Exception as e:
        time.sleep(5)
        print(f"Error de conexión: {e}")
        GenerarImagen(prompt,size)

    url=respuesta['data'][0]['url']

    return url

app = gr.Interface(fn=GenerarImagen,inputs=[
     gr.Textbox("Una imagen de un gato al estilo manga y cyberpunk",label="Prompt"),
     gr.Dropdown(["256x256", "512x512", "1024x1024"],value="256x256",label="Size")
     ], outputs=gr.Image(label="Imagen generada"))


app.queue()

app.launch()
```

# Transcripción de audio

```python
import openai
import time
import requests
from io import BytesIO
import gradio as gr

# Obtiene la clave de API desde las variables de entorno
key = os.getenv("OPENAI_API_KEY")

# Asigna la clave de API a la variable api_key del módulo openai
openai.api_key = key

def TranscribirAudio(url):
    print(url)

    audio_file = open(url,"rb")

    try:
        transcript = openai.Audio.transcribe("whisper-1", audio_file)

    except Exception as e:
        time.sleep(5)
        print(f"Error de conexión: {e}")
        TranscribirAudio(url)

    return transcript['text']

app = gr.Interface(fn=TranscribirAudio,inputs=[
    gr.Audio(label="Audio", source="upload", type="filepath")],
                   outputs=gr.Textbox(label="Transcripción"))
app.launch()

```

# Generador de presentaciones
 

```python
#Consulta a la API del chat
import openai
import time
import os
import json

 

from dotenv import load_dotenv
 
 
load_dotenv()


key = os.getenv("OPENAI_API_KEY")

openai.api_key = key

MensajeDSistema = "Eres parte de un sistema de generacion de diapositivas, cuando recibas un tema debes retornar\
    una lista en formato json que tenga 5 diapositivas con titulo,contenido que sera definicion del titulo y \
        descripcion detallada de  una imagen en ingles que sera utilizada en un generador de imagenes,\
            los nombres de las claves seran, titulo,contenido y descripcion_imagen.\
            Ejemplo: {\"diapositivas\": [ {\"titulo\": \"Introducción a la Ofimática\", \"contenido\":\
        \"La ofimática es el conjunto de herramientas, técnicas y programas informáticos que se utilizan en la gestión\
            y automatización de tareas de oficina. Incluye aplicaciones como procesadores de texto, hojas de cálculo,\
                programas de presentación, bases de datos, entre otros.\", \"descripcion_imagen\": \"An image showing \
                    a person working on a computer with various office software icons on the screen.\"}],}    "
            
 
consulta = "reposteria"#@param {type:"string"}
temperatura = 0

mensajes = [{"role": "system", "content": MensajeDSistema},
            {"role": "user", "content": consulta} ]

def LLM( mensajes, temp):
    try:
        resultado = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
          messages=mensajes,
          temperature=temp)
    except Exception as e :
        time.sleep(5)
        print(f"Error de conexión: {e}")
        LLM(mensajes, temp)
    return resultado["choices"][0]["message"]["content"]

diapositivas=LLM(mensajes=mensajes,temp=temperatura)
diapositivas = json.loads(diapositivas) 

```

```python

# Generador de Imagenes

from PIL import Image
import requests

def GenerarImagen(prompt,size="256x256"):
    try:
        respuesta=  openai.Image.create(
            prompt=prompt,
            n=1,
            size=size)
    except Exception as e:
        time.sleep(5)
        print(f"Error de conexión: {e}")
        GenerarImagen(prompt,size)

    url=respuesta['data'][0]['url']
    imagen=Image.open(requests.get(url, stream=True).raw)
    return imagen


 

def ObtenerImagen(textoimagen,id=0):
    tmp=GenerarImagen(prompt=textoimagen)
    ruta=f"../tmp/{id}.png"
    tmp.save(ruta)
     
    return ruta

```

```python
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.text import MSO_ANCHOR, PP_ALIGN

def dividir_texto(texto, max_chars):
    palabras = texto.split(' ')
    lineas = []
    linea = ''
    for palabra in palabras:
        if len(linea + ' ' + palabra) <= max_chars:
            linea += ' ' + palabra
        else:
            lineas.append(linea)
            linea = palabra
    lineas.append(linea)
    return '\n'.join(lineas)

# Crear una presentación
presentation = Presentation()

# Agregar una diapositiva con un diseño de título y contenido
slide_layout = presentation.slide_layouts[1]

for diapositiva in diapositivas['diapositivas'] :
    
    slide = presentation.slides.add_slide(slide_layout)
    title = slide.shapes.title
    title.text = diapositiva['titulo']
    title.text_frame.paragraphs[0].font.size = Inches(0.6) 

    # Agregar una imagen a la diapositiva
    imagen = ObtenerImagen(diapositiva['descripcion_imagen'])
     
    
    img_width = Inches(4)  # Ancho de la imagen
    img_height = Inches(4)  # Altura de la imagen
    left = (presentation.slide_width - img_width) / 2  # Centrar horizontalmente
    top = (presentation.slide_height - img_height) / 2  # Centrar verticalmente
    slide.shapes.add_picture(imagen, left, top, width=img_width, height=img_height)

    # Agregar el texto "content" debajo de la imagen en una fuente pequeña
    #content_left = (presentation.slide_width - img_width) / 2  # Centrar horizontalmente
    content_left = Inches(0.1) 
    content_top = top + img_height - Inches(0.2)
    content_width = presentation.slide_width - Inches(0)  # Asegurar que el cuadro de texto quede dentro de la diapositiva
    content_height = presentation.slide_height - content_top - Inches(0.1)  # Asegurar que el cuadro de texto quede dentro de la diapositiva

    content = slide.shapes.add_textbox(content_left, content_top, content_width, content_height)
    text_frame = content.text_frame
    text_frame.textAnchor = MSO_ANCHOR.MIDDLE  # Centrar verticalmente el texto en el cuadro de texto

    p = text_frame.add_paragraph()
    p.text = dividir_texto(diapositiva['contenido'], 60) 
    p.font.size = Inches(0.3)  # Tamaño de fuente pequeño
    p.alignment = PP_ALIGN.CENTER  # Centrar horizontalmente el texto en el cuadro de texto
    break     
# Guardar la presentación
presentation.save("../tmp/presentacion.pptx")

```